In [1]:
# connect to google drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
FOLDERNAME = 'AI\ poster'
%cd drive/MyDrive/$FOLDERNAME

Mounted at /content/drive
/content/drive/.shortcut-targets-by-id/1RS_eU17m1jbzKezF_6yS85UbgONfLHtY/AI poster


In [ ]:
# device
import torch, os
if torch.cuda.is_available():
  device = torch.device('cuda')
else:
  device = torch.device('cpu')
print('Device:', device)
print(os.getcwd())

Device: cuda
/content/drive/.shortcut-targets-by-id/1RS_eU17m1jbzKezF_6yS85UbgONfLHtY/AI poster


In [ ]:
import shutil
train_path = '/content/drive/MyDrive/AI poster/PA/ASVspoof2019_PA_asv_train'
os.chdir(train_path+'/wav_train')

for png in os.listdir(train_path+'/wav_train/spoof_1'):
  shutil.move(train_path+'/wav_train/spoof_1/'+png, train_path+'/wav_train/spoof')

In [ ]:
print(len(os.listdir(train_path+'/wav_train/bonafide')))

0


In [ ]:
%pip install pydub
%pip install torchaudio

# for data preproccessing
from pathlib import PurePath
from pydub import AudioSegment
import shutil

# for loading data from drive
from torch.utils.data import DataLoader
import torchaudio.datasets as dset

# for creating waveform
import numpy as np
import soundfile as sf
import scipy.signal as signal
import matplotlib.pyplot as plt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
train_path = '/content/drive/MyDrive/AI poster/PA/ASVspoof2019_PA_asv_train'

os.chdir(train_path)

In [ ]:
os.mkdir(train_path+'/wav_train_30000')
os.mkdir(train_path+'/wav_train_30000/spoof')
os.mkdir(train_path+'/wav_train_30000/bonafide')

os.mkdir(train_path+'/wav_val_30000')
os.mkdir(train_path+'/wav_val_30000/spoof')
os.mkdir(train_path+'/wav_val_30000/bonafide')

In [ ]:
import os
print(os.getcwd())
train_path = '/content/drive/MyDrive/AI poster/PA/ASVspoof2019_PA_asv_train'
os.chdir(train_path)

print(len(os.listdir(train_path+'/wav_train_30000/spoof')))
print(len(os.listdir(train_path+'/wav_train_30000/bonafide')))
print(len(os.listdir(train_path+'/wav_val_30000/spoof')))
print(len(os.listdir(train_path+'/wav_val_30000/bonafide')))


/content/drive/MyDrive/AI poster/PA/ASVspoof2019_PA_asv_train/wav_val/spoof
9000
5507
6001
2937


In [ ]:
%pip install pydub
%pip install torchaudio

# for data preproccessing
from pathlib import PurePath
from pydub import AudioSegment
import shutil

# for loading data from drive
from torch.utils.data import DataLoader
import torchaudio.datasets as dset

# for creating waveform
import numpy as np
import soundfile as sf
import scipy.signal as signal
import matplotlib.pyplot as plt

# labeling
labels={}
with open('/content/drive/MyDrive/AI poster/PA/ASVspoof2019_PA_cm_protocols/ASVspoof2019.PA.cm.train.trn.txt','r') as f:
  for line in f:
    labels[line.split()[1]]=line.split()[-1]
print(labels)

wav_names = os.listdir('/content/drive/.shortcut-targets-by-id/1RS_eU17m1jbzKezF_6yS85UbgONfLHtY/AI poster/PA_T/ASVspoof2019_PA_train/wav')
wav_names=sorted(wav_names, reverse=True)
print(wav_names[500])

train_path = '/content/drive/MyDrive/AI poster/PA/ASVspoof2019_PA_asv_train'
PA_T_wav_path = '/content/drive/.shortcut-targets-by-id/1RS_eU17m1jbzKezF_6yS85UbgONfLHtY/AI poster/PA_T/ASVspoof2019_PA_train/wav'

import wave


for wav_name in wav_names:
  if not os.path.exists(train_path+'/wav_train/spoof/'+wav_name[:len(wav_name)-4]+'.png') and not os.path.exists(train_path+'/wav_val/spoof/'+wav_name[:len(wav_name)-4]+'.png') and not os.path.exists(train_path+'/wav_val/bonafide/'+wav_name[:len(wav_name)-4]+'.png') and not os.path.exists(train_path+'/wav_train/bonafide/'+wav_name[:len(wav_name)-4]+'.png'):
    os.chdir(PA_T_wav_path)

    wav_obj = wave.open(PA_T_wav_path+'/'+wav_name, 'rb')

    freq = wav_obj.getframerate() # Hz per sec
    n_freq = wav_obj.getnframes() # total Hz
    time = n_freq/freq      # lengh of the audio(s)
    # print(f'audio time: {time}')

    # channel check (1 channel)
    n_channels = wav_obj.getnchannels()
    # print(f'channel: {n_channels}')

    signal_wave = wav_obj.readframes(n_freq)
    signal_array = np.frombuffer(signal_wave, dtype=np.int16)

    # create waveform
    times = np.linspace(0, time, num=n_freq)
    plt.figure(figsize=(15, 5))
    plt.plot(times, signal_array)
    plt.title(wav_name)
    plt.xlabel('time(s)')
    plt.xlim(0, time)
    plt.ylabel('signal value')

    # save img (40% val 60% train)
    if int(wav_name[11]) < 4:
      if labels[wav_name[:12]] == 'bonafide':
        os.chdir(train_path+'/wav_val/bonafide')
      else:
        os.chdir(train_path+'/wav_val/spoof')
      plt.savefig(wav_name[:len(wav_name)-4]+'.png')

    else:
      if labels[wav_name[:12]] == 'bonafide':
        os.chdir(train_path+'/wav_train/bonafide')
      else:
        os.chdir(train_path+'/wav_train/spoof')
      plt.savefig(wav_name[:len(wav_name)-4]+'.png')

    print(wav_name)

In [ ]:
import torchvision.datasets as dset
import torchvision.transforms as T

transform = T.Compose([T.RandomHorizontalFlip(p=1),T.Resize((224,224))])

train = dset.ImageFolder('wav_train', transform=transform)
val = dset.ImageFolder('wav_val', transform=transform)

In [ ]:
from torch.utils.data import DataLoader
mini_trains = DataLoader(train, batch_size=50, shuffle=True)
mini_vals = DataLoader(val, batch_size=50, shuffle=True)

In [ ]:
from torchvision import models
import torch.nn as nn
import torch.optim as optim

resnet = models.resnet50(pretrained=True).cuda()
num_flatten = resnet.fc.in_features
resnet.fc = nn.Linear(num_flatten, 2)
model = resnet.to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-4)

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

In [ ]:
NUM_EPOCH = 3
PRINT_EVERY = 1000

In [ ]:
model.summary()

In [ ]:
def evaluate(model, mini_vals):
  model.eval()
  with torch.no_grad():
    total_acc = 0
    for x, y in mini_vals:
      x = x.to(device)
      y = y.to(device)
      scores = model(x)
      predictions = scores.max(1)[1]
      acc = predictions.eq(y).sum().item()
      total_acc += acc
    print('acc: ' ,total_acc / len(val))

In [ ]:
for epoch in range(NUM_EPOCH):
  for counter, (x, y) in enumerate(mini_trains):
    model.train()
    x = x.to(device)
    y = y.to(device)
    scores = model(x)
    loss_function = nn.CrossEntropyLoss()
    loss = loss_function(scores, y)
    if counter % PRINT_EVERY == 0:
      print(f'epoch {epoch+1} :{loss.item()}', end='/')
      evaluate(model, mini_vals)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

epoch 1 :0.6961867809295654/acc:  0.3291592128801431
epoch 2 :0.0002911660703830421/acc:  1.0
epoch 3 :9.980234608519822e-05/acc:  1.0
